<a href="https://colab.research.google.com/github/ArnaudOlt/dissertation_pybamm/blob/main/4_2_temp_cap_loss_over_throughput_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pybtex
%pip install anytree
%pip install autograd
%pip install bpx
%pip install casadi
%pip install imageio
%pip install importlib-metadata
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install scikit-fem
%pip install scipy
%pip install sympy
%pip install tqdm
%pip install xarray
%pip install git+https://github.com/pybamm-team/pybamm.git@develop
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as plt

In [ ]:
model = pybamm.lithium_ion.SPMe(
    {
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # alias for "SEI porosity change"
    }
)

In [ ]:
prada2013_params = pybamm.ParameterValues("Prada2013")

def plating_exchange_current_density_OKane2020(c_e, c_Li, T):
    """
    Exchange-current density for Li plating reaction [A.m-2].
    References
    ----------
    .. [1] O’Kane, Simon EJ, Ian D. Campbell, Mohamed WJ Marzook, Gregory J. Offer, and
    Monica Marinescu. "Physical origin of the differential voltage minimum associated
    with lithium plating in Li-ion batteries." Journal of The Electrochemical Society
    167, no. 9 (2020): 090540.
    Parameters
    ----------
    c_e : :class:`pybamm.Symbol`
        Electrolyte concentration [mol.m-3]
    c_Li : :class:`pybamm.Symbol`
        Plated lithium concentration [mol.m-3]
    T : :class:`pybamm.Symbol`
        Temperature [K]
    Returns
    -------
    :class:`pybamm.Symbol`
        Exchange-current density [A.m-2]
    """

    k_plating = pybamm.Parameter("Lithium plating kinetic rate constant [m.s-1]")

    return pybamm.constants.F * k_plating * c_e


def stripping_exchange_current_density_OKane2020(c_e, c_Li, T):
    """
    Exchange-current density for Li stripping reaction [A.m-2].

    References
    ----------

    .. [1] O’Kane, Simon EJ, Ian D. Campbell, Mohamed WJ Marzook, Gregory J. Offer, and
    Monica Marinescu. "Physical origin of the differential voltage minimum associated
    with lithium plating in Li-ion batteries." Journal of The Electrochemical Society
    167, no. 9 (2020): 090540.

    Parameters
    ----------

    c_e : :class:`pybamm.Symbol`
        Electrolyte concentration [mol.m-3]
    c_Li : :class:`pybamm.Symbol`
        Plated lithium concentration [mol.m-3]
    T : :class:`pybamm.Symbol`
        Temperature [K]

    Returns
    -------

    :class:`pybamm.Symbol`
        Exchange-current density [A.m-2]
    """

    k_plating = pybamm.Parameter("Lithium plating kinetic rate constant [m.s-1]")

    return pybamm.constants.F * k_plating * c_Li

def SEI_limited_dead_lithium_OKane2022(L_sei):
    """
    Decay rate for dead lithium formation [s-1].
    References
    ----------
    .. [1] Simon E. J. O'Kane, Weilong Ai, Ganesh Madabattula, Diega Alonso-Alvarez,
    Robert Timms, Valentin Sulzer, Jaqueline Sophie Edge, Billy Wu, Gregory J. Offer
    and Monica Marinescu. "Lithium-ion battery degradation: how to model it."
    Physical Chemistry: Chemical Physics 24, no. 13 (2022): 7909-7922.
    Parameters
    ----------
    L_sei : :class:`pybamm.Symbol`
        Total SEI thickness [m]
    Returns
    -------
    :class:`pybamm.Symbol`
        Dead lithium decay rate [s-1]
    """

    gamma_0 = pybamm.Parameter("Dead lithium decay constant [s-1]")
    L_inner_0 = pybamm.Parameter("Initial inner SEI thickness [m]")
    L_outer_0 = pybamm.Parameter("Initial outer SEI thickness [m]")
    L_sei_0 = L_inner_0 + L_outer_0

    gamma = gamma_0 * L_sei_0 / L_sei

    return gamma



prada2013_params.update({
        #experiment
        "Lower voltage cut-off [V]": 2.5,
        "Upper voltage cut-off [V]": 3.65,
        #cell
        "Negative current collector thickness [m]": 0.000006,
        "Separator thickness [m]": 0.000016,
        "Positive current collector thickness [m]": 0.000013,
        "Electrode height [m]": 0.325,
        "Electrode width [m]": 21.6,
        "Cell cooling surface area [m2]": 0.07338004,
        "Cell volume [m3]": 0.00106398976,
        "Positive current collector density [kg.m-3]": 2700,
        "Positive current collector thermal conductivity [W.m-1.K-1]": 237,
        "Nominal cell capacity [A.h]":100,
        "Current function [A]":100,
        "Contact resistance [Ohm]": 0.0004,
        #electrolyte
        "Electrolyte conductivity [S.m-1]": 0.97,



        #Okane2022 Li plating
        "Lithium metal partial molar volume [m3.mol-1]": 1.3e-05,
        "Lithium plating kinetic rate constant [m.s-1]": 1e-09,
        "Exchange-current density for plating [A.m-2]": plating_exchange_current_density_OKane2020,
        "Exchange-current density for stripping [A.m-2]": stripping_exchange_current_density_OKane2020,
        "Initial plated lithium concentration [mol.m-3]": 0.0,
        "Typical plated lithium concentration [mol.m-3]": 1000.0,
        "Lithium plating transfer coefficient": 0.65,
        "Dead lithium decay constant [s-1]": 1e-06,
        "Dead lithium decay rate [s-1]": SEI_limited_dead_lithium_OKane2022,
        #Okane2022 SEI
        "Ratio of lithium moles to SEI moles": 1.0,
        "Inner SEI reaction proportion": 0.0,
        "Inner SEI partial molar volume [m3.mol-1]": 9.585e-05,
        "Outer SEI partial molar volume [m3.mol-1]": 9.585e-05,
        "SEI reaction exchange current density [A.m-2]": 1.5e-07,
        "SEI resistivity [Ohm.m]": 200000.0,
        "Outer SEI solvent diffusivity [m2.s-1]": 2.5000000000000002e-22,
        "Bulk solvent concentration [mol.m-3]": 2636.0,
        "Inner SEI open-circuit potential [V]": 0.1,
        "Outer SEI open-circuit potential [V]": 0.8,
        "Inner SEI electron conductivity [S.m-1]": 8.95e-14,
        "Inner SEI lithium interstitial diffusivity [m2.s-1]": 1e-20,
        "Lithium interstitial reference concentration [mol.m-3]": 15.0,
        "Initial inner SEI thickness [m]": 0.0,
        "Initial outer SEI thickness [m]": 5e-09,
        "EC initial concentration in electrolyte [mol.m-3]": 4541.0,
        "EC diffusivity [m2.s-1]": 2e-18,
        "SEI kinetic rate constant [m.s-1]": 1e-12,
        "SEI open-circuit potential [V]": 0.4,
        "SEI growth activation energy [J.mol-1]": 38000.0,
        "Negative electrode reaction-driven LAM factor [m3.mol-1]": 0.0,
        "Positive electrode reaction-driven LAM factor [m3.mol-1]": 0.0

        }, check_already_exists=False)


In [ ]:
# standard cycling experiment
pybamm.set_logging_level("NOTICE")
cycle_number = 500
exp = pybamm.Experiment(
    [("Discharge at 1C until 2.5 V",  # ageing cycles
    "Charge at 0.3C until 3.65 V",
    "Hold at 3.65 V until C/20",
    "Rest for 4 hours",)] * cycle_number
)

In [ ]:
temperatures = [268.15, 278.15,  288.15] #-5, 5, 15°C

solutions_standard = {} # dict, key is temp and values is solution object

for temp in temperatures:
    param = prada2013_params
    param.update({"Ambient temperature [K]": temp})

    sim = pybamm.Simulation(model, parameter_values=param, experiment=exp)
    safe_solver = pybamm.CasadiSolver(mode="safe", dt_max=120)
    sol = sim.solve(solver=safe_solver, calc_esoh=False)
    solutions_standard[temp] = sol
    #solutions_standard[temp].save(f"solutions_standard_cycle_{temp}.pkl")


In [ ]:
# Lunana cycling experiment
pybamm.set_logging_level("NOTICE")
cycle_number = 500
exp_lunana = pybamm.Experiment(
    [("Discharge at 0.02C until 2.5 V",  # ageing cycles
    "Charge at 0.08C until 3.65 V",
    "Hold at 3.65 V until C/20",
    "Rest for 4 hours",)] * cycle_number
)

In [ ]:
temperatures = [268.15, 278.15, 288.15] #-5, 5, 15°C

solutions_lunana = {} # dict, key is temp and values is solution object

for temp in temperatures:
    param = prada2013_params
    param.update({"Ambient temperature [K]": temp})

    sim = pybamm.Simulation(model, parameter_values=param, experiment=exp_lunana)
    safe_solver = pybamm.CasadiSolver(mode="safe", dt_max=120)
    sol = sim.solve(solver=safe_solver, calc_esoh=False)
    solutions_lunana[temp] = sol
    #solutions_lunana[temp].save(f"solutions_lunana_cycle_{temp}.pkl")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
plt.rcParams.update({'font.size': 12})
line_colors = ["lightblue", "coral", "crimson"]
cycle_numbers = sol.summary_variables["Cycle number"]
label_names_standard = ["Temperature = -5°C, standard cycling (higher C-rate)", "Temperature = 5°C, standard cycling (higher C-rate)", "Temperature = 15°C, standard cycling (higher C-rate)", "Temperature = 14.3 °C"]
label_names_lunana = ["Temperature = -5°C, adjusted cycling (lower C-rate)", "Temperature = 5°C, adjusted cycling (lower C-rate)", "Temperature = 15°C, adjusted cycling (lower C-rate)", "Temperature = 14.3 °C"]


# Loop through the solutions and their corresponding temperatures
for i, (temperature, sol) in enumerate(solutions_standard.items()):
    color = line_colors[i % len(line_colors)]
    label = label_names_standard[i % len(label_names_standard)]  # Cycle through label_names if needed

    x_value = []
    y_value = []

    cycle_start_throughput = 0

    for cycle in cycle_numbers-1:
        throughput_cycle = sol.cycles[cycle]["Throughput capacity [A.h]"].data
        capacity_cycle = sol.cycles[cycle]["Loss of capacity to lithium plating [A.h]"].data
        min_index = np.argmin(capacity_cycle)

        capacity_value = capacity_cycle[min_index]
        corresponding_throughput_value = throughput_capacity[min_index]
        x_value.append(cycle_start_throughput + corresponding_throughput_value)
        y_value.append(capacity_value)
        cycle_start_throughput += throughput_cycle[-1]



    ax.plot(x_value, y_value, linestyle='-', color=color, alpha=0.7, label = label)

# Loop through the solutions and their corresponding temperatures
for i, (temperature, sol) in enumerate(solutions_lunana.items()):
    color = line_colors[i % len(line_colors)]
    label = label_names_lunana[i % len(label_names_lunana)]  # Cycle through label_names if needed

    x_value = []
    y_value = []

    cycle_start_throughput = 0

    for cycle in cycle_numbers-1:
        throughput_cycle = sol.cycles[cycle]["Throughput capacity [A.h]"].data
        capacity_cycle = sol.cycles[cycle]["Loss of capacity to lithium plating [A.h]"].data
        min_index = np.argmin(capacity_cycle)

        capacity_value = capacity_cycle[min_index]
        corresponding_throughput_value = throughput_capacity[min_index]
        x_value.append(cycle_start_throughput + corresponding_throughput_value)
        y_value.append(capacity_value)
        cycle_start_throughput += throughput_cycle[-1]



    ax.plot(x_value, y_value, linestyle='dashed', color=color, alpha=0.7, label = label)


# Add labels and legend
ax.legend(fontsize = 11)
ax.set_xlabel("Throughput capacity [A.h]", fontsize=14)
ax.set_ylabel("Capacity loss [A.h]", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)


# Adjust layout and display the plot
plt.tight_layout()



In [ ]:
# Comparing different C rates over 500 cycles

x_axis_var_str = "Time [h]"
#x_axis_var_str = "Throughput capacity [A.h]"
y_axis_var_str = "Loss of capacity to lithium plating [A.h]"


fig, ax = plt.subplots(figsize=(8, 6))

line_colors = ["lightblue", "coral", "crimson", "cyan", "magenta", "yellow"]
label_names_standard = ["Temperature = -5°C, standard cycling", "Temperature = 5°C, standard cycling", "Temperature = 15°C, standard cycling", "Temperature = 14.3 °C"]
label_names_lunana = ["Temperature = -5°C, adjusted cycling", "Temperature = 5°C, adjusted cycling", "Temperature = 15°C, adjusted cycling", "Temperature = 14.3 °C"]


for i, (temp, sol) in enumerate(list(solutions_standard.items())[:3]):
    x_sol = sol.summary_variables[x_axis_var_str]
    y_sol = sol.summary_variables[y_axis_var_str]

    color = line_colors[i % len(line_colors)]  # Cycle through colors if needed
    label = label_names_standard[i % len(label_names_standard)]  # Cycle through label_names if needed

    ax.plot(x_sol, y_sol, label=label, color = color)

for i, (temp, sol) in enumerate(list(solutions_lunana.items())[:3]):
    x_sol = sol.summary_variables[x_axis_var_str]
    y_sol = sol.summary_variables[y_axis_var_str]

    color = line_colors[i % len(line_colors)]  # Cycle through colors if needed
    label = label_names_lunana[i % len(label_names_lunana)]  # Cycle through label_names if needed
    ax.plot(x_sol, y_sol, label=label, color = color, linestyle="--")


ax.set_xlabel(x_axis_var_str, fontsize=14)
ax.set_ylabel("Loss of capacity to lithium plating [A.h]", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
ax.legend(fontsize = 13)

plt.tight_layout()

plt.show()